# 2018 NUS-MIT Datathon Tutorial: Machine Learning on CBIS-DDSM

## Goal

In this colab, we are going to train a simple [convolutional neural network (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network) with Tensorflow, which can be used to classify the mammographic images based on [breast density](https://www.cancercenter.com/discussions/blog/breast-density-becoming-an-important-predictor-of-breast-cancer-risk/).

The network we are going to build is adapted from the [official tensorflow tutorial](https://www.tensorflow.org/tutorials/layers).

## CBIS-DDSM

The dataset we are going to work with is [CBIS-DDSM](https://wiki.cancerimagingarchive.net/display/Public/CBIS-DDSM). Quote from their website:

> "This CBIS-DDSM (Curated Breast Imaging Subset of DDSM) is an updated and standardized version of the Digital Database for Screening Mammography (DDSM)."

CBIS-DDSM differs from the [original DDSM dataset](http://marathon.csee.usf.edu/Mammography/Database.html) in that it converted images to [DICOM](https://en.wikipedia.org/wiki/DICOM) format, which is easier to work with.

Note that although this tutorial focuses on the CBIS-DDSM dataset, most of it can be easily applied to [The International Skin Imaging Collaboration (ISIC)](https://isic-archive.com/) dataset as well. More details will be provided in the Datasets section below.

## Setup



To be able to run the code cells in this tutorial, you need to create a copy of this Colab notebook by clicking "File" > "Save a copy in Drive..." menu.

You can share your copy with your teammates by clicking on the "SHARE" button on the top-right corner of your Colab notebook copy. Everyone with "Edit" permission is able to modify the notebook at the same time, so it is a great way for team collaboration.

First Let's import modules needed to complete the tutorial. You can run the following cell by clicking on the triangle button when you hover over the [ ] space on the top-left corner of the code cell below.

In [1]:
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import glob
import SimpleITK as sitk
import cv2


from io import BytesIO
from matplotlib import pyplot
# The next import is used to print out pretty pandas dataframes
from IPython.display import display, HTML
from PIL import Image

You can verify that GPU is working with the following code cell.

In [ ]:
# Should output something like '/device:GPU:0'.
tf.test.gpu_device_name()

## Dataset

We have already extracted the images from the DICOM files to [separate folders](http://storage.cloud.google.com/datathon-cbis-ddsm-colab) on GCS, and some preprocessing were also done with the raw images (If you need custom preprocessing, please consult our [tutorial on image preprocessing](http://colab.research.google.com/github/GoogleCloudPlatform/healthcare/blob/master/datathon/nusdatathon18/tutorials/image_preprocessing.ipynb)).

The folders ending with `_demo` contain subsets of training and test images. Specifically, the demo training dataset has 100 images, with 25 images for each [breast density category](https://breast-cancer.ca/densitbi-rads/) (1 - 4). There are 20 images in the test dataset which were selected randomly. All the images were first padded to 5251x7111 (largest width and height among the selected images) and then resized to 95x128 to fit in memory and save training time. Both training and test images are ["Cranial-Caudal"](https://breast-cancer.ca/mammopics/) only.

[ISIC dataset](http://storage.cloud.google.com/isic-images) is organized in a slightly different way, the images are in JPEG format and each image comes with a JSON file containing metadata information. In order to make this tutorial work for ISIC, you will need to first pad and resize the images (we provide a script to do that [here](https://github.com/GoogleCloudPlatform/healthcare/tree/master/datathon/nusdatathon18/scripts)), and extract the labels from the JSON files based on your interests.

## Training

Before coding on our neurual network,  let's create a few helper methods to make loading data from Google Cloud Storage (GCS) easier.

In [ ]:
def load_img_label(train = True):
    if train:
        df = pd.read_csv(os.getcwd() + "/../CBIS-DDSM-classic/calc_case_description_train_set.csv")
    else:
        df = pd.read_csv(os.getcwd() + "/../CBIS-DDSM-classic/calc_case_description_test_set.csv")
        
    df = df[df["breast density"] != 0]
    img_dir =  os.getcwd() + "/../CBIS-DDSM-classic/" + df["image file path"]
    label = df["breast density"]
    
    return img_dir.values, label.values


def load_images(train = True):
    
    images = []
    labels = []
    
    img_dirs, _labels = load_img_label(train) 
    idx = np.arange(0 , int(len(img_dirs)/15))
    np.random.shuffle(idx)
    img_dirs = [img_dirs[i] for i in idx]
    _labels = [_labels[i] for i in idx]
    blob = zip(img_dirs, _labels)
       
    for img_dir, _label in blob:
        
        img = sitk.ReadImage(img_dir)
        img = sitk.GetArrayFromImage(img)
        img = np.array(img, dtype= np.float32)
        img = np.squeeze(img)/10000
        img = cv2.resize(img, dsize = (95,128))
        images.append(img)
        labels.append(_label-1)
        
    return np.array(images), np.array(labels, dtype=np.int32)

def load_train_images():
    return load_images(train = True)

def load_test_images():
    return load_images(train = False)

Let's create a model function, which will be passed to an estimator that we will create later. The model has an architecture of 6 layers:

1.   *Convolutional Layer*: Applies 32 5x5 filters, with ReLU activation function
2.   *Pooling Layer*: Performs max pooling with a 2x2 filter and stride of 2
3.   *Convolutional Layer*: Applies 64 5x5 filters, with ReLU activation function
4.   *Pooling Layer*: Same setup as #2
5.   *Dense Layer*: 1,024 neurons,  with dropout regulartization rate of 0.25
6.   *Logits Layer*: 4 neurons, one for each breast density category, i.e. [0, 4)

Note that you can change the parameters on the right (or inline) to tune the neurual network. It is **highly recommended** to check out the original [tensorflow tutorial](https://www.tensorflow.org/tutorials/layers) to get a deeper understanding of the network we are building here. 

In [ ]:
KERNEL_SIZE = 5 #@param
DROPOUT_RATE = 0.25 #@param

def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""

  # Input Layer.
  # Reshape to 4-D tensor: [batch_size, height, width, channels]
  # DDSM images are grayscale, which have 1 channel.
  input_layer = tf.reshape(features["x"], [-1, 95, 128, 1])

  # Convolutional Layer #1.
  # Input Tensor Shape: [batch_size, 95, 128, 1]
  # Output Tensor Shape: [batch_size, 95, 128, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=KERNEL_SIZE,
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1.
  # Input Tensor Shape: [batch_size, 95, 128, 1]
  # Output Tensor Shape: [batch_size, 47, 64, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2.
  # Input Tensor Shape: [batch_size, 47, 64, 32]
  # Output Tensor Shape: [batch_size, 47, 64, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=KERNEL_SIZE,
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2.
  # Input Tensor Shape: [batch_size, 47, 64, 32]
  # Output Tensor Shape: [batch_size, 23, 32, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 23, 32, 64]
  # Output Tensor Shape: [batch_size, 23 * 32 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 23 * 32 * 64])

  # Dense Layer.
  # Input Tensor Shape: [batch_size, 25 * 17 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Dropout operation.
  # 0.75 probability that element will be kept.
  dropout = tf.layers.dropout(inputs=dense, rate=DROPOUT_RATE,
                              training=(mode == tf.estimator.ModeKeys.TRAIN))

  # Logits Layer.
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 4]
  logits = tf.layers.dense(inputs=dropout, units=4)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Loss Calculation.
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode).
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

Now that we have a model function, next step is feeding it to an estimator for training. Here are are creating a main function as required by tensorflow.

In [ ]:
BATCH_SIZE = 20 #@param
STEPS = 10000 #@param

def main(_):
  # Load training and test data.
  train_data, train_labels = load_train_images()
  eval_data, eval_labels = load_test_images()

  # Create the Estimator.
  ddsm_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn)

  # Set up logging for predictions.
  # Log the values in the "Softmax" tensor with label "probabilities".
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model.
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=BATCH_SIZE,
      num_epochs=None,
      shuffle=True)
  ddsm_classifier.train(
      input_fn=train_input_fn,
      steps=STEPS,
      hooks=[logging_hook])

  # Evaluate the model and print results.
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data},
      y=eval_labels,
      num_epochs=1,
      shuffle=False)
  eval_results = ddsm_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)

Finally, here comes the exciting moment. We are going to train and evaluate the model we just built! Run the following code cell and pay attention to the accuracy printed at the end of logs.

Note if this is **not** the first time you run the following cell, to avoid weird errors like "NaN loss during training", please run the following command to remove the temporary files.

In [ ]:
# Set logging level.
tf.logging.set_verbosity(tf.logging.INFO)

# Start training, this will call the main method defined above behind the scene.
# The whole training process will take ~5 mins.
tf.app.run()